In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
dataset =  pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train.csv')
dataset.head(10)

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1722.0,32575.0,Prague_1_2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1688.0,32507.0,Prague_1_2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1696.0,32552.0,Prague_1_2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,0,0.0,0,0.8,0.0,1681.0,32423.0,Prague_1_2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,0,0.0,0,0.5,0.0,1704.0,32410.0,Prague_1_2020-12-09
5,Prague_1,2020-12-10,7228.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1740.0,32432.0,Prague_1_2020-12-10
6,Prague_1,2020-12-11,7790.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1730.0,32471.0,Prague_1_2020-12-11
7,Prague_1,2020-12-12,7165.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1708.0,32476.0,Prague_1_2020-12-12
8,Prague_1,2020-12-13,6844.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1717.0,32458.0,Prague_1_2020-12-13
9,Prague_1,2020-12-14,7010.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1723.0,32505.0,Prague_1_2020-12-14


In [3]:
testset =  pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test.csv')
testset.head(10)

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,Prague_1,2024-03-16,NaN,0,0,0,0,Prague_1_2024-03-16
1,Prague_1,2024-03-17,NaN,0,0,0,0,Prague_1_2024-03-17
2,Prague_1,2024-03-18,NaN,0,0,0,0,Prague_1_2024-03-18
3,Prague_1,2024-03-19,NaN,0,0,0,0,Prague_1_2024-03-19
4,Prague_1,2024-03-20,NaN,0,0,0,0,Prague_1_2024-03-20
5,Prague_1,2024-03-21,NaN,0,0,0,0,Prague_1_2024-03-21
6,Prague_1,2024-03-22,NaN,0,0,0,0,Prague_1_2024-03-22
7,Prague_1,2024-03-23,NaN,0,0,0,0,Prague_1_2024-03-23
8,Prague_1,2024-03-24,NaN,0,0,0,0,Prague_1_2024-03-24
9,Prague_1,2024-03-25,NaN,0,0,0,0,Prague_1_2024-03-25


In [4]:
dataset['holiday_name'] = dataset['holiday_name'].fillna(0)
dataset['precipitation'] = dataset['precipitation'].fillna(dataset['precipitation'].mean())
dataset['snow'] = dataset['snow'].fillna(0)

In [5]:
testset['holiday_name'] = testset['holiday_name'].fillna(0)

In [6]:
dataset['date'] = pd.to_datetime(dataset['date'])
dataset['date'] = dataset['date'].dt.day_name()


In [7]:
testset['date'] = pd.to_datetime(testset['date'])
testset['date'] = testset['date'].dt.day_name()

In [8]:
warehouse = {'Prague_1': 1,
'Brno_1':2,
'Prague_2':3,
'Prague_3':4,
'Budapest_1':5,
'Munich_1':6,
'Frankfurt_1':7}

testset['warehouse'] = testset['warehouse'].map(warehouse)
dataset['warehouse'] = dataset['warehouse'].map(warehouse)

date = {'Wednesday': 1,
'Thursday':2,
'Saturday':3,
'Tuesday':4,
'Friday':5,
'Monday':6,
'Sunday':7}

dataset['date'] = dataset['date'].map(date)
testset['date'] = testset['date'].map(date)

dataset.head(10)
testset.head(10)

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,1,3,0,0,0,0,0,Prague_1_2024-03-16
1,1,7,0,0,0,0,0,Prague_1_2024-03-17
2,1,6,0,0,0,0,0,Prague_1_2024-03-18
3,1,4,0,0,0,0,0,Prague_1_2024-03-19
4,1,1,0,0,0,0,0,Prague_1_2024-03-20
5,1,2,0,0,0,0,0,Prague_1_2024-03-21
6,1,5,0,0,0,0,0,Prague_1_2024-03-22
7,1,3,0,0,0,0,0,Prague_1_2024-03-23
8,1,7,0,0,0,0,0,Prague_1_2024-03-24
9,1,6,0,0,0,0,0,Prague_1_2024-03-25


In [9]:
dataset = dataset.drop(['id', 'shutdown', 'mini_shutdown', 'blackout', 'mov_change', 
                        'frankfurt_shutdown', 'precipitation', 'snow', 'user_activity_1',
                       'user_activity_2'], axis=1)
#dataset.head()

In [10]:
X = dataset.drop('orders', axis=1)
y = dataset['orders']

train_X, test_X, train_y, test_y = train_test_split(X, y,test_size= 0.1, random_state=42)

print(train_X)
print(train_y)

      warehouse  date holiday_name  holiday  shops_closed  \
6483          5     3            0        0             0   
1487          2     7            0        0             0   
1263          2     7            0        0             0   
156           1     4            0        0             0   
5866          7     3            0        0             0   
...         ...   ...          ...      ...           ...   
5191          6     6            0        0             0   
5226          6     6            0        0             0   
5390          6     6            0        0             0   
860           1     6            0        0             0   
7270          5     5            0        0             0   

      winter_school_holidays  school_holidays  
6483                       0                0  
1487                       0                0  
1263                       0                0  
156                        0                0  
5866                       

In [11]:
train_X['holiday_name'].unique()

array([0, 'Christmas Eve', 'International womens day', 'Cyrila a Metodej',
       'Memorial Day for the Victims of the Communist Dictatorships',
       'Den ceske statnosti', 'New Years Day', 'Labour Day',
       '2nd Christmas Day', 'Den boje za svobodu a demokracii',
       'Den vzniku samostatneho ceskoslovenskeho statu', 'Jan Hus',
       'Independent Hungary Day', 'Den osvobozeni', 'Good Friday',
       'Easter Monday', 'Reformation Day', "All Saints' Day Holiday",
       'Memorial Day of the Republic', 'Day of National Unity',
       'National Defense Day',
       '1848 Revolution Memorial Day (Extra holiday)',
       'Peace Festival in Augsburg',
       'Memorial Day for the Victims of the Holocaust',
       'Memorial Day for the Martyrs of Arad'], dtype=object)

In [12]:
holiday_name = {
"International womens day": 1,
"Christmas Eve": 2,
"2nd Christmas Day": 3,
"Good Friday": 4,
"New Years Day": 5,
"Den osvobozeni": 6,
"Easter Monday": 7,
"Den ceske statnosti": 8,
"Labour Day": 9,
"Cyrila a Metodej": 10,
"Jan Hus": 11,
"Den vzniku samostatneho ceskoslovenskeho statu": 12,
"Den boje za svobodu a demokracii": 13,
"Memorial Day of the Republic": 14,
"Independent Hungary Day": 15,
"Day of National Unity": 16,
"Reformation Day": 17,
"National Defense Day": 18,
"Memorial Day for the Victims of the Holocaust": 19,
"Memorial Day for the Martyrs of Arad": 20,
"Memorial Day for the Victims of the Communist Dictatorships": 21,
"All Saints' Day Holiday": 22,
"1848 Revolution Memorial Day (Extra holiday)": 23,
"Peace Festival in Augsburg": 24}

train_X['holiday_name'] = train_X['holiday_name'].map(holiday_name)
train_X.fillna(0, inplace=True)
test_X['holiday_name'] = test_X['holiday_name'].map(holiday_name)
test_X.fillna(0, inplace=True)

testset['holiday_name'] = testset['holiday_name'].map(holiday_name)
testset.fillna(0, inplace=True)

print(train_X['holiday_name'].value_counts())
print(test_X['holiday_name'].value_counts())


holiday_name
0.0     6415
1.0       22
2.0       19
3.0       15
5.0       14
6.0       12
4.0       12
10.0      11
8.0       11
12.0      11
7.0       11
9.0       10
13.0      10
11.0       9
14.0       4
16.0       3
15.0       3
21.0       3
17.0       2
18.0       2
19.0       2
20.0       2
22.0       1
23.0       1
24.0       1
Name: count, dtype: int64
holiday_name
0.0     707
1.0       4
2.0       4
11.0      3
13.0      2
9.0       2
5.0       2
18.0      1
12.0      1
7.0       1
20.0      1
3.0       1
21.0      1
10.0      1
8.0       1
24.0      1
19.0      1
Name: count, dtype: int64


In [13]:
test_id = testset['id']
test = testset.drop('id', axis=1)

In [14]:
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
print(train_X)

[[ 0.71285611 -0.45894947 -0.13977038 ... -0.10643704 -0.17301595
  -0.08373885]
 [-0.88961467  1.58750357 -0.13977038 ... -0.10643704 -0.17301595
  -0.08373885]
 [-0.88961467  1.58750357 -0.13977038 ... -0.10643704 -0.17301595
  -0.08373885]
 ...
 [ 1.24701304  1.07589031 -0.13977038 ... -0.10643704 -0.17301595
  -0.08373885]
 [-1.4237716   1.07589031 -0.13977038 ... -0.10643704 -0.17301595
  -0.08373885]
 [ 0.71285611  0.56427705 -0.13977038 ... -0.10643704 -0.17301595
  -0.08373885]]


In [15]:
scaler = StandardScaler()
test_data = scaler.fit_transform(test)
print(test_data)

[[-1.44293436 -0.45444366 -0.19042803 ... -0.14340695 -0.14340695
  -0.27546462]
 [-1.44293436  1.60155787 -0.19042803 ... -0.14340695 -0.14340695
  -0.27546462]
 [-1.44293436  1.08755748 -0.19042803 ... -0.14340695 -0.14340695
  -0.27546462]
 ...
 [ 0.59929359  1.08755748 -0.19042803 ... -0.14340695 -0.14340695
  -0.27546462]
 [ 0.59929359  0.05955672 -0.19042803 ... -0.14340695 -0.14340695
  -0.27546462]
 [ 0.59929359 -1.48244443 -0.19042803 ... -0.14340695 -0.14340695
  -0.27546462]]


In [16]:
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
decision_tree = DecisionTreeRegressor(random_state=42)

ensemble_model = VotingRegressor(
    estimators=[
    ('rf', random_forest),
    ('dt', decision_tree)
])

model = ensemble_model.fit(train_X, train_y)

In [17]:
scaler = StandardScaler()
test_X = scaler.fit_transform(test_X)
print(test_X)

[[-1.34737278  0.59934347 -0.1543397  ... -0.09078413 -0.19914833
  -0.09078413]
 [-1.34737278  0.09252537 -0.1543397  ... -0.09078413 -0.19914833
  -0.09078413]
 [ 0.24346629  1.10616157  3.13610591 ... 11.01514109 -0.19914833
  -0.09078413]
 ...
 [-1.34737278  0.59934347 -0.1543397  ... -0.09078413  5.02138285
  -0.09078413]
 [ 1.30402567  0.09252537 -0.1543397  ... -0.09078413 -0.19914833
  -0.09078413]
 [-0.81709309 -0.92111082 -0.1543397  ... -0.09078413 -0.19914833
  -0.09078413]]


In [18]:
y_pred = ensemble_model.predict(test_X)
mape = mean_absolute_percentage_error(test_y, y_pred)
print(f'Voting Regressor MAPE: {mape:.4f}')

Voting Regressor MAPE: 0.1167


In [19]:
pred = ensemble_model.predict(test)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


In [20]:
submission = pd.DataFrame({
    'id': test_id,
    'orders': pred
})

# Save
submission.to_csv('submission.csv', index=False)

print(submission)

                        id       orders
0      Prague_1_2024-03-16  1383.306047
1      Prague_1_2024-03-17  1383.306047
2      Prague_1_2024-03-18  1383.306047
3      Prague_1_2024-03-19  1383.306047
4      Prague_1_2024-03-20  3565.813190
..                     ...          ...
392  Budapest_1_2024-05-11  1528.486137
393  Budapest_1_2024-05-12  1528.486137
394  Budapest_1_2024-05-13  1528.486137
395  Budapest_1_2024-05-14  1528.486137
396  Budapest_1_2024-05-15  1528.486137

[397 rows x 2 columns]
